In [132]:
from string import ascii_lowercase
import random
from itertools import combinations
import numpy as np

In [133]:
def creatproblem(n,k,m):
        positive_var = list(ascii_lowercase)[:n]
        negative_var = [var.upper() for var in positive_var]
        variables = positive_var + negative_var
        problem = []
        threshold = 10
        i = 0
        comb = list(combinations(variables,k))

        while i<threshold:
            c = random.sample(comb,m)
            if c not in problem:
                problem.append(c)
                i += 1

        problems_new = []
        for c in problem:
            temp = []
            temp = [list(sub) for sub in c]
            problems_new.append(temp)
        return  variables,problems_new

In [134]:
def random_assign(variables,n):
    litral = list(np.random.choice(2,n))
    negation = [abs(i-1) for i in litral]
    assign = litral + negation
    return dict(zip(variables,assign))

In [135]:
def heruistic(problem,assign):
    count = 0
    for sub in problem:
        encode = [assign[val] for val in sub]
        count += any(encode)
    return count

In [136]:
def next_node(current):
    key = list(current.keys())
    key.sort()
    key = key[0:len(key)//2]
    successors = []
    for k in key:
        temp = current.copy()
        temp[k] = abs(temp[k]-1)
        temp[chr(ord(k)+32)] = abs(temp[chr(ord(k)+32)]-1)
        successors.append(temp)

    return successors
print(next_node({'a': 0, 'b': 0, 'c': 0, 'A': 1, 'B': 1, 'C': 1}))

[{'a': 1, 'b': 0, 'c': 0, 'A': 0, 'B': 1, 'C': 1}, {'a': 0, 'b': 1, 'c': 0, 'A': 1, 'B': 0, 'C': 1}, {'a': 0, 'b': 0, 'c': 1, 'A': 1, 'B': 1, 'C': 0}]


In [137]:
def select_node(succs,problem):
    heruistic_val = []
    for i in succs:
        heruistic_val.append(heruistic(problem,i))
    index = heruistic_val.index(max(heruistic_val))
    return succs[index]

In [138]:
def check_goal_state(state,problem):
    count = 0
    for sub in problem:
        encode = [state[val] for val in sub]
        count += any(encode)
    return len(problem) == count

In [139]:
def hill_climbing(current_state,problem,step):
    if step == 10000:
        print('halted at depth',step)
        return
    if check_goal_state(current_state,problem)==True:
        print(current_state,step)
        return
    else:
        step += 1
        successor = next_node(current_state)
        new_node = select_node(successor,problem)
        hill_climbing(new_node,problem,step)

In [140]:
n = 2
k = 3
m = 2
var,prob = creatproblem(n,k,m)
print(var)
for i in prob:
    print(i)

['a', 'b', 'A', 'B']
[['a', 'A', 'B'], ['a', 'b', 'A']]
[['a', 'b', 'B'], ['a', 'b', 'A']]
[['a', 'A', 'B'], ['a', 'b', 'B']]
[['a', 'b', 'B'], ['a', 'A', 'B']]
[['a', 'b', 'A'], ['a', 'b', 'B']]
[['b', 'A', 'B'], ['a', 'b', 'A']]
[['a', 'b', 'A'], ['a', 'A', 'B']]
[['b', 'A', 'B'], ['a', 'b', 'B']]
[['a', 'A', 'B'], ['b', 'A', 'B']]
[['b', 'A', 'B'], ['a', 'A', 'B']]


In [141]:
start_state = {k: int(v) for k, v in start_state.items()}
print(start_state,prob[0])
# start_state = {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f':0,'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 1, 'F':1}
hill_climbing(start_state,prob[0],0)

{'a': 0, 'b': 1, 'c': 0, 'd': 0, 'e': 0, 'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1} [['a', 'A', 'B'], ['a', 'b', 'A']]
{'a': 0, 'b': 1, 'c': 0, 'd': 0, 'e': 0, 'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1} 0


In [142]:
print(check_goal_state({'a': 0, 'b': 1, 'c': 0, 'd': 1, 'A': 1, 'B': 0, 'C': 1, 'D': 0},prob[7]))

True


Beam Search

In [143]:
def heruistic(problem,assign):
    count = 0
    for sub in problem:
        encode = [assign[val] for val in sub]
        count += any(encode)
    return count

In [144]:
def select_node_beam(succs,problem,beamwidth):
    heruistic_val = []
    beam_nodes = []
    for i in succs:
        heruistic_val.append(heruistic(problem,i))
    for i in range(beamwidth):
        index = heruistic_val.index(max(heruistic_val))
        beam_nodes.append(succs[index])
        succs.remove(succs[index])

    return beam_nodes

In [145]:
def check_goal_state(state,problem,beamwidth):
    count = 0
    for sub in problem:
        encode = [state[val] for val in sub]
        count += any(encode)
    return len(problem) == count

In [146]:
def next_node_beam(succs):
    nextnode= []
    for current in succs:
        key = list(current.keys())
        key.sort()
        key = key[0:len(key)//2]
        successors = []
        for k in key:
            temp = current.copy()
            temp[k] = abs(temp[k]-1)
            temp[chr(ord(k)+32)] = abs(temp[chr(ord(k)+32)]-1)
            successors.append(temp)
            nextnode.append(temp)

    return nextnode

In [147]:
def beam_search(start,problem,step,beamwidth):
    if step == 10000:
        print('halted at depth',step)
        return
    for current_state in start:
        print(current_state)
        if check_goal_state(current_state,problem,beamwidth)==True:
            print(current_state,step)
            return
    step +=1
    successor = next_node_beam(start,problem,beamwidth)
    new_node = select_node_beam(successor,prob[0],beamwidth)
    beam_search(start,problem,step,beamwidth)

In [148]:
n = 5
k = 3
m = 5
var,probl = creatproblem(n,k,m)
print(var)
for i in probl:
    print(i)

['a', 'b', 'c', 'd', 'e', 'A', 'B', 'C', 'D', 'E']
[['e', 'A', 'E'], ['c', 'e', 'C'], ['b', 'e', 'A'], ['a', 'e', 'A'], ['b', 'A', 'C']]
[['c', 'D', 'E'], ['A', 'D', 'E'], ['b', 'c', 'C'], ['a', 'b', 'd'], ['a', 'A', 'E']]
[['d', 'A', 'E'], ['b', 'D', 'E'], ['d', 'A', 'D'], ['a', 'e', 'A'], ['c', 'e', 'C']]
[['a', 'C', 'D'], ['d', 'C', 'D'], ['B', 'D', 'E'], ['a', 'c', 'C'], ['d', 'e', 'D']]
[['e', 'A', 'B'], ['a', 'b', 'A'], ['b', 'e', 'E'], ['a', 'B', 'E'], ['a', 'A', 'C']]
[['B', 'D', 'E'], ['c', 'B', 'D'], ['d', 'e', 'B'], ['d', 'e', 'A'], ['b', 'c', 'E']]
[['b', 'd', 'A'], ['e', 'A', 'E'], ['b', 'B', 'E'], ['c', 'A', 'C'], ['d', 'A', 'E']]
[['e', 'B', 'D'], ['d', 'e', 'B'], ['c', 'A', 'B'], ['a', 'e', 'A'], ['a', 'c', 'e']]
[['e', 'A', 'C'], ['a', 'd', 'e'], ['d', 'C', 'D'], ['a', 'd', 'B'], ['a', 'd', 'D']]
[['b', 'd', 'D'], ['b', 'e', 'C'], ['d', 'A', 'D'], ['b', 'C', 'D'], ['d', 'A', 'E']]


In [149]:

beam_search([start_state,start_state],prob[2],0,1)

{'a': 0, 'b': 1, 'c': 0, 'd': 0, 'e': 0, 'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1}
{'a': 0, 'b': 1, 'c': 0, 'd': 0, 'e': 0, 'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1} 0


VAriable neighbourhood descent with 3 descent

In [150]:
def heruistic(problem,assign):
    count = 0
    for sub in problem:
        encode = [assign[val] for val in sub]
        count += any(encode)
    return count

In [151]:
def check_goal_state(state,problem):
    count = 0
    for sub in problem:
        encode = [state[val] for val in sub]
        count += any(encode)
    return len(problem) == count

In [152]:
def select_node(succs,problem):
    heruistic_val = []
    for i in succs:
        heruistic_val.append(heruistic(problem,i))
    index = heruistic_val.index(max(heruistic_val))
    return succs[index]

In [153]:
def nghd1(current):
    key = list(current.keys())
    key.sort()
    key = key[0:len(key)//2]
    successors = []
    for k in key:
        temp = current.copy()
        temp[k] = abs(temp[k]-1)
        temp[chr(ord(k)+32)] = abs(temp[chr(ord(k)+32)]-1)
        successors.append(temp)


    return successors

In [154]:

print(nghd1({'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}))

[{'a': 0, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 1, 'B': 0, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 0, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1}]


In [155]:
def nghd2(current):
    key = list(current.keys())
    key.sort()
    key = key[0:len(key)//2]
    successors = []
    for j in range(0,len(key)-1):
        for i in range(j,len(key)-1):

            temp = current.copy()
            temp[key[i]] = abs(temp[key[i]]-1)
            temp[key[i+1]] = abs(temp[key[i+1]]-1)
            temp[chr(ord(key[i])+32)] = abs(temp[chr(ord(key[i])+32)]-1)
            temp[chr(ord(key[i+1])+32)] = abs(temp[chr(ord(key[i+1])+32)]-1)
            successors.append(temp)


    return successors
print(nghd2({'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}))

[{'a': 0, 'b': 0, 'c': 1, 'd': 1, 'e': 1, 'A': 1, 'B': 1, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}]


In [156]:
def nghd3(current):
    key = list(current.keys())
    key.sort()
    key = key[0:len(key)//2]
    successors = []
    for j in range(0,len(key)-2):
        for k in range(j,len(key)-2):
            for i in range(k,len(key)-2):

                temp = current.copy()
                temp[key[i]] = abs(temp[key[i]]-1)
                temp[key[i+1]] = abs(temp[key[i+1]]-1)
                temp[chr(ord(key[i])+32)] = abs(temp[chr(ord(key[i])+32)]-1)
                temp[chr(ord(key[i+1])+32)] = abs(temp[chr(ord(key[i+1])+32)]-1)
                successors.append(temp)


    return successors
print(nghd3({'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}))

[{'a': 0, 'b': 0, 'c': 1, 'd': 1, 'e': 1, 'A': 1, 'B': 1, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}]


In [157]:
def Variable_Neighborhood1(current_state,problem,step):
    if step == 10000:
        print('halted at depth',step)
        return
    if check_goal_state(current_state,problem)==True:
        return current_state

    else:
        step += 1
        successor = next_node(current_state)
        new_node = nghd1(successor,problem)
        Variable_Neighborhood(new_node,problem,step)

In [158]:
def Variable_Neighborhood2(current_state,problem,step):
    if step == 10000:
        print('halted at depth',step)
        return
    if check_goal_state(current_state,problem)==True:
        return current_state

    else:
        step += 1
        successor = next_node(current_state)
        new_node = nghd2(successor,problem)
        Variable_Neighborhood(new_node,problem,step)

In [159]:
def Variable_Neighborhood3(current_state,problem,step):
    if step == 10000:
        print('halted at depth',step)
        return
    if check_goal_state(current_state,problem)==True:
        print(current_state,step)
        return
    else:
        step += 1
        successor = next_node(current_state)
        new_node = nghd3(successor,problem)
        Variable_Neighborhood(new_node,problem,step)

In [160]:
n = 5
k = 3
m = 5
var,probl = creatproblem(n,k,m)
print(var)
for i in probl:
    print(i)

['a', 'b', 'c', 'd', 'e', 'A', 'B', 'C', 'D', 'E']
[['A', 'D', 'E'], ['b', 'C', 'E'], ['c', 'e', 'E'], ['b', 'e', 'D'], ['b', 'c', 'D']]
[['c', 'e', 'B'], ['c', 'd', 'A'], ['A', 'B', 'C'], ['c', 'd', 'D'], ['c', 'B', 'E']]
[['c', 'C', 'E'], ['a', 'b', 'c'], ['b', 'd', 'e'], ['C', 'D', 'E'], ['e', 'A', 'D']]
[['a', 'd', 'D'], ['d', 'e', 'C'], ['b', 'B', 'D'], ['c', 'C', 'E'], ['b', 'c', 'B']]
[['b', 'd', 'D'], ['b', 'e', 'E'], ['c', 'e', 'B'], ['a', 'd', 'D'], ['b', 'C', 'D']]
[['a', 'b', 'D'], ['b', 'B', 'E'], ['c', 'd', 'C'], ['c', 'A', 'D'], ['e', 'C', 'E']]
[['b', 'C', 'D'], ['a', 'c', 'e'], ['a', 'd', 'E'], ['c', 'A', 'D'], ['c', 'C', 'E']]
[['b', 'A', 'E'], ['b', 'A', 'C'], ['a', 'b', 'e'], ['c', 'd', 'B'], ['d', 'A', 'B']]
[['b', 'c', 'C'], ['c', 'C', 'E'], ['e', 'C', 'D'], ['a', 'e', 'A'], ['b', 'A', 'D']]
[['b', 'D', 'E'], ['c', 'e', 'A'], ['d', 'D', 'E'], ['c', 'd', 'e'], ['d', 'e', 'C']]


In [161]:
start_state = random_assign(var,n)
print([start_state,start_state])

[{'a': np.int64(1), 'b': np.int64(1), 'c': np.int64(0), 'd': np.int64(0), 'e': np.int64(0), 'A': np.int64(0), 'B': np.int64(0), 'C': np.int64(1), 'D': np.int64(1), 'E': np.int64(1)}, {'a': np.int64(1), 'b': np.int64(1), 'c': np.int64(0), 'd': np.int64(0), 'e': np.int64(0), 'A': np.int64(0), 'B': np.int64(0), 'C': np.int64(1), 'D': np.int64(1), 'E': np.int64(1)}]


In [163]:
var1 = start_state
var2 = Variable_Neighborhood1(var1,prob[0],0)
var3 = Variable_Neighborhood2(var2,prob[0],0)
Variable_Neighborhood3(var3,prob[0],0)

{'a': np.int64(1), 'b': np.int64(1), 'c': np.int64(0), 'd': np.int64(0), 'e': np.int64(0), 'A': np.int64(0), 'B': np.int64(0), 'C': np.int64(1), 'D': np.int64(1), 'E': np.int64(1)} 0
